In [2]:
import pickle
from tensorflow import keras
import tensorflow
# Cofigures the gpus
gpus_list = tensorflow.config.list_physical_devices('GPU')
if gpus_list:
    try:
        for gpu in gpus_list:
            tensorflow.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

def read_file(path):
    with open(path, "rb") as file:
        data = pickle.load(file)
        x_train = data["x_train"]
        y_train = data["y_train"]
        x_test = data["x_test"]
        y_test = data["y_test"]
        vocab_size = data["vocab_size"]
        max_length = data["max_length"]
    return x_train, y_train, x_test, y_test, vocab_size, max_length

def padde(x_train, x_test, max_length):
    x_train_padded = keras.utils.pad_sequences(x_train, maxlen=max_length)
    x_test_padded = keras.utils.pad_sequences(x_test, maxlen=max_length)
    return x_train_padded, x_test_padded

x_train, y_train, x_test, y_test, vocab_size, max_length = read_file("/content/keras-data.pickle")

x_train, x_test = padde(x_train, x_test, max_length)

print(vocab_size)
print(max_length)


1000
1062


In [3]:
import numpy as np

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [4]:
print("x_train shape: ", x_train.shape)
print("y_train length:", len(y_train))
print("x_test shape: ", x_test.shape)
print("y_test length:", len(y_test))

x_train shape:  (393053, 1062)
y_train length: 393053
x_test shape:  (130528, 1062)
y_test length: 130528


In [5]:
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)


In [6]:
recurrent_model = keras.Sequential()
recurrent_model.add(keras.layers.Embedding(vocab_size+1, 64))
recurrent_model.add(keras.layers.LSTM(64))
recurrent_model.add(keras.layers.Dense(units=num_classes, activation="sigmoid"))

In [7]:
optim = keras.optimizers.Adam(learning_rate=1e-3)
recurrent_model.compile(optimizer=optim, loss="binary_crossentropy", metrics=["accuracy"])

In [8]:
recurrent_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
recurrent_model.fit(x_train, y_train, epochs=1)
loss, accuracy = recurrent_model.evaluate(x_test, y_test)
print("loss: ", loss)
print("accuracy: ", accuracy)

12283/12283 ━━━━━━━━━━━━━━━━━━━━ 400s 32ms/step - accuracy: 0.8976 - loss: 0.2485
4079/4079 ━━━━━━━━━━━━━━━━━━━━ 59s 14ms/step - accuracy: 0.9386 - loss: 0.1536
loss:  0.15187233686447144
accuracy:  0.9400588274002075
